In [25]:
from bs4 import BeautifulSoup
import requests
#import bs4
import lxml
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import numpy as np
import re

In [6]:
url_booking = 'https://www.tripadvisor.com.ph/Restaurants-g298573-Manila_Metro_Manila_Luzon.html'
headers_booking = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9'
}

response_booking  = requests.get(url_booking, headers=headers_booking)

In [8]:
soup_booking = BeautifulSoup(response_booking.text, 'html.parser')
#print(soup_booking)

In [9]:
restaurants_name_booking = soup_booking.select('div', {'class': 'QEXGj'})
restaurants_name_booking

[<div id="fb-root"></div>,
 <div class="ppr_rup ppr_priv_sync_rt_cookie" data-placement-name="sync_rt_cookie" id="taplc_sync_rt_cookie_0">
 <script type="text/javascript">require(["ta/Core/TA.Store"], function(taStore) {taStore.store("retargeting.rtURL", '//' + 'www.tamgrt.com/RT');taStore.store("retargeting.geoId", '298573');});</script><script type="text/javascript">require(["@ta/platform.runtime"], function (runtime) {runtime.importBundle("@ta/platform.consent").then(function (bundle) {bundle.requestConsent(bundle.CategoriesEnum.TARGETING, function () {(function(window, document) {"use strict";var SYNC_AGE = 1000*60*30; var WHITELIST_AGE = 1000*60*60*24; var HTTP_COOKIE = "TART";var SYNC_COOKIE = "SRT";var WHITELIST_COOKIE = "WLRedir";var MSG_PREFIX = "COOKIESYNC:";var _isNewRTCookie = true;var _tart = 'enc%3AC43824pw80typXWmzikYg3Oe88D%2Bh1fHdW6FivtFndPQ%2FxXFOKTXGXHkIKEiaoU%2B8ZzwB7Grm10%3D';if (_hasCookie(HTTP_COOKIE) && !_isNewRTCookie) { return; } var syncFrame = document.creat

In [42]:
restaurants_data_booking = []
# Loop over the hotel elements and extract the desired data
for restaurant_booking in restaurants_name_booking:
    # Extract the name
    restaurant_name_booking_element = restaurant_booking.select_one('a', {'class': 'Lwqic Cj b'})
    restaurant_name_booking = None
    if restaurant_name_booking_element:
        restaurant_name_booking = restaurant_name_booking_element.text.strip()

    # Extract the location
    restaurant_location_booking_element = restaurant_booking.find('div', {'class': False })
    restaurant_location_booking = None
    if restaurant_location_booking_element:
        restaurant_location_booking = restaurant_location_booking_element.text.strip()  
    #svg={'class': 'UctUV d H0'}, 
    # Extract the hotel review score
    css_selector = 'svg.UctUV.d.H0[aria-label]'
    restaurant_reviewscore_booking_element = restaurant_booking.select_one(css_selector)
    restaurant_reviewscore_booking = None
    if restaurant_reviewscore_booking_element:
        restaurant_reviewscore_booking = restaurant_reviewscore_booking_element.text.strip()
    if restaurant_reviewscore_booking_element:
        # Get the value of the aria-label attribute
        aria_label = restaurant_reviewscore_booking_element["aria-label"]
        #print("aria-label:", aria_label)
        restaurant_reviewscore_booking = aria_label
    #else:
        #print("Element with aria-label attribute not found.")
    
    # Extract the hotel review feedback
    restaurant_reviewfeedback_booking_element = restaurant_booking.find('span', {'class': 'tvdtW'})
    restaurant_reviewfeedback_booking = None
    if restaurant_reviewfeedback_booking_element:
        restaurant_reviewfeedback_booking = restaurant_reviewfeedback_booking_element.text.strip()
    
    # Extract the hotel review count
    restaurant_reviewcount_booking_element = restaurant_booking.find('span', {'class': 'IiChw'})
    restaurant_reviewcount_booking = None
    if restaurant_reviewcount_booking_element:
        restaurant_reviewcount_booking = restaurant_reviewcount_booking_element.text.strip()
    
    # Extract the open
    restaurant_open_booking_element = restaurant_booking.find('span', {'class': 'JjSWRd'})
    restaurant_open_booking = None
    if restaurant_open_booking_element:
        restaurant_open_booking = restaurant_open_booking_element.text.strip()
    
    # Extract the menu
    #restaurant_menu_booking_element = restaurant_booking.find('a', {'class': 'xFAlBc'})
    #restaurant_menu_booking = None
    #if restaurant_menu_booking_element:
        #restaurant_menu_booking = restaurant_menu_booking_element.text.strip()

    # Append hotes_data with info about hotel
    restaurants_data_booking.append({
        'Restaurant_Name': restaurant_name_booking,
        'Restaurant_Location': restaurant_location_booking,        
        'Restaurant_Review_Score': restaurant_reviewscore_booking,
        'Restaurant_Review_Feedback': restaurant_reviewfeedback_booking,
        'Restaurant_Review_Count': restaurant_reviewcount_booking,
        #'Restaurant_OpenHours': restaurant_open_booking_element,
        #'Restaurant_Menu': restaurant_menu_booking
    })

In [43]:

restaurants_booking = pd.DataFrame(restaurants_data_booking)
restaurants_booking = restaurants_booking.dropna()
restaurants_booking.drop(restaurants_booking[restaurants_booking['Restaurant_Name'] == ''].index, inplace=True)
restaurants_booking.drop(restaurants_booking[restaurants_booking['Restaurant_Location'] == ''].index, inplace=True)
restaurants_booking = restaurants_booking.drop_duplicates()
restaurants_booking['Restaurant_Name'] = restaurants_booking['Restaurant_Name'].str.replace(r'^\d+\.\s', '', regex=True)
restaurants_booking['Restaurant_Location'] = 'Manila'
restaurants_booking['Restaurant_Review_Score'] = restaurants_booking['Restaurant_Review_Score'].str.replace(" of 5 bubbles", "").astype(float)
restaurants_booking['Restaurant_Review_Count'] = restaurants_booking['Restaurant_Review_Count'].str.replace(" reviews", "")
restaurants_booking['Restaurant_Review_Count'] = restaurants_booking['Restaurant_Review_Count'].str.replace(",", "").astype(int)
restaurants_booking.head()

,Restaurant_Name,Restaurant_Location,Restaurant_Review_Score,Restaurant_Review_Feedback,Restaurant_Review_Count
227,Le Bar,Metro Manila,4.5,Making celebrations more special,290
253,Cafe Ilang-Ilang,Metro Manila,4.5,"... fruit and salad, Korean, Chinese, e...",582
279,Eng Bee Tin,Metro Manila,4.5,Culinary delight in Chinatown,238
306,Sky Deck at The Bayleaf Hotel,Metro Manila,4.0,Very slow service,611
332,Harbor View Restaurant,Metro Manila,4.0,"I did order a Salmon Pasta, but also ta...",584


In [38]:
restaurants_booking.to_csv('C:/Users/mneme/Desktop/Python Data Engineering files/Restaurant.csv', header=True, index=False)